In [ ]:
from dask.distributed import Client
import numpy as np
import os, tifffile,sys, glob, cv2
import dask.array as da
import dask
from tqdm import tqdm
from skimage.io import imread
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import matplotlib.pyplot as plt
from skimage import data, io
from math import ceil, floor
%matplotlib inline
sys.path.append('../../Preprocessing')
from preproc import beautify_frame

# Start a client using all CPUs of this machine
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56317,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:56330,Total threads: 3
Dashboard: http://127.0.0.1:56331/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:56320,


In [2]:
# In this approach I will store img_gray and img_pp as global variables. Here I don't use Dask. It doesn't seem to work, we repeat the same 5 frames over and over again.

imgs = {}        # np.array of original images
gray_imgs = {}  # np.array of grayscale images
pp_imgs = {}    # np.array of preprocessed images

# First we read the images in Dask
images_folder = "/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/h1r1_1minute/"
# Fetch all files in the folder
all_files = glob.glob(images_folder + "*.jpg")

# Reads an image
def read_image(idx):
    global imgs
    # Add the image to the dictionary
    imgs[idx] = cv2.imread(all_files[idx])

# Converts an image to grayscale and adds it the the gray_imgs dictionary
def convert_gray(idx):
    gray_imgs[idx] = cv2.cvtColor(imgs[idx], cv2.COLOR_BGR2GRAY)

def preprocess(idx):
    if idx not in gray_imgs:
        raise Exception('Attempt to pp an image that is not in gray_imgs')
    pp_imgs[idx] = beautify_frame(gray_imgs[idx])

# Function to apply filtering on a substack of images
def percentile_custom(filter_range, percentile=75):
    # Check that there are no missing images in the filter_range in pp_imgs
    if len([i for i in filter_range if i not in pp_imgs]) > 0:
        raise Exception('Attempt to filter a substack that contains missing images in pp_imgs')
    
    # Return the percentile across the specified axis
    substack = [pp_imgs[i] for i in filter_range]
    return np.percentile(substack, percentile, axis=0).astype(np.uint8)

start_index = 0  # Included
end_index = 10000    # Excluded
step = 120           # We filter every step images
filter_length = 40  # Number of images to consider for the filter
percentile = 75     # Percentile to take for the filter
frame_skip = 2      # Frames skipped between each image for the filter

filtered_imgs = []
for idx in tqdm(range(start_index, end_index, step), desc="Processing images"):
    filter_idxs = [i for i in range(max(start_index, idx-floor(filter_length/2)), min(end_index, idx+ceil(filter_length/2)))]
    # Find the images between idx-floor(filter_length/2) and idx+ceil(filter_length/2) that are missing from pp_imgs
    missing_pp = [i for i in filter_idxs if i not in pp_imgs]
    for idx_no_pp in missing_pp:
        # If the image is not in gray_imgs, do the conversion
        if idx_no_pp not in gray_imgs:
            # Read the image if it has not yet been read
            if idx_no_pp not in imgs:
                read_image(idx_no_pp)
            convert_gray(idx_no_pp)
        # Preprocess the image
        preprocess(idx_no_pp)

    # Now that we have all pictures in pp_imgs, we can apply the filter
    filtered_imgs.append(percentile_custom(filter_idxs, percentile))
    # Delete the images in imgs, gray_imgs and pp_imgs that have not been used in the filter at this iteration. That is images that are not in filter_idxs
    for key in [k for k in imgs if k not in filter_idxs]:
        del imgs[key]
    for key in [k for k in gray_imgs if k not in filter_idxs]:
        del gray_imgs[key]
    for key in [k for k in pp_imgs if k not in filter_idxs]:
        del pp_imgs[key]
    

# Save the filtered images as a tiff stack in Results/Cyril/
filtered_imgs = np.array(filtered_imgs)
tifffile.imwrite('Results/Cyril/filtered_imgs.tif', filtered_imgs)


Processing images: 100%|██████████| 84/84 [15:49<00:00, 11.30s/it]


In [14]:
# Different approach: For each filtered image, compute the needed substack of pp images and apply the filter
# First we read the images in Dask
#images_folder = "/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/h1r1_1minute/"
images_folder = "/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/24.09-24.10_observation_OH/Images/h1r1_1minute/"
all_files = os.path.join(images_folder, '*.jpg')
images = dask_image.imread.imread(all_files)
print("Dask images: ", images)

# Prepare all filenames
filenames = os.listdir(images_folder)
filenames.sort()
# Remove hidden files
filenames = [f for f in filenames if not f.startswith('.')]

# Read the first image to get the dimensions
first_image = cv2.imread(os.path.join(images_folder, filenames[0]))
height, width, _ = first_image.shape
print("Image dimensions: ", height, width)

# Converts an image to grayscale
def convert_gray_old(bgr): # 20min33
    result = ((bgr[..., 0] * 0.114) +
              (bgr[..., 1] * 0.587) +
              (bgr[..., 2] * 0.299))
    return result

@delayed
def convert_gray(img): # 13min14
    np_img = np.array(img) # Convert from Dask array to numpy array
    return cv2.cvtColor(np_img, cv2.COLOR_BGR2GRAY)

# Function to apply filtering on a substack of images
@delayed
def percentile_custom(substack, percentile=75):
    # Return the percentile across the specified axis
    return np.percentile(substack, percentile, axis=0).astype(np.uint8)

@delayed
def annotate_name(img, filename):
    # Pre-calculate parameters for annotation
    font_scale = 2
    font_color = (255, 0, 0)
    thickness = 3
    position = (10, 70)  # Fixed position for text
    return cv2.putText(img, filename, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, thickness, cv2.LINE_AA)

@delayed
def beautify_frame_delayed(img):
    #img = np.array(img) # Convert from Dask array to numpy array
    return beautify_frame(img)

def filter_substack(i,filter_length,percentile,frame_skip=1):
    # Get the substack
    substack = images[max(0,i-frame_skip*floor(filter_length/2)):i + frame_skip*ceil(filter_length/2)]
    # Convert the substack to grayscale
    substack_gray = [convert_gray(img) for img in substack]
    # Preprocess the substack
    substack_gray = [beautify_frame_delayed(img) for img in substack_gray]
    # Calculate the percentile
    percentile_img = percentile_custom(substack_gray, percentile)
    return percentile_img

def percentile_filter(start_idx, stop_idx,step=1,frame_skip=1,filter_length=40,percentile=75):
    '''
    This function makes a percentile filter of images between start and stop indexes.
    '''
    idxs = range(start_idx, stop_idx, step) # Images that need to be filtered
    filtered_imgs = [filter_substack(i,filter_length,percentile,frame_skip) for i in idxs]
    # Annotate all images with their name
    filtered_imgs = [annotate_name(img, filenames[idx]) for idx, img in zip(idxs,filtered_imgs)]
    filtered_imgs = da.stack([da.from_delayed(d, shape=(height,width), dtype=np.uint8) for d in filtered_imgs], axis=0)
    return filtered_imgs

# filter substack parameters
filter_length = 40
percentile = 90
frame_skip = 2
step = 180

filtered_imgs = percentile_filter(len(images)-10000, len(images)-1, step, frame_skip, filter_length, percentile)

print("Filtered images: ", filtered_imgs)
results = filtered_imgs.compute()

# Save the results in a temporary folder as a single tiff file
tifffile.imwrite(f'Results/Cyril/fl{filter_length}_p{percentile}_fs{frame_skip}_step{step}.tiff', results)

Dask images:  dask.array<_map_read_frame, shape=(110775, 2592, 4608, 3), dtype=uint8, chunksize=(1, 2592, 4608, 3), chunktype=numpy.ndarray>
Image dimensions:  2592 4608
Filtered images:  dask.array<stack, shape=(56, 2592, 4608), dtype=uint8, chunksize=(1, 2592, 4608), chunktype=numpy.ndarray>


KeyboardInterrupt: 

In [ ]:
# In this approach I compute all gray images and then all pp images and then all filtered images

# First we read the images in Dask
images_folder = "/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/h1r1_1minute/"
all_files = os.path.join(images_folder, '*.jpg')
images = dask_image.imread.imread(all_files)
# Only keep the first 1000 images
images = images[:1000]
print(images)
height, width = images.shape[1:3]

# Converts an image to grayscale
@delayed
def convert_gray(img):
    img_np = np.array(img)
    return cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)

# Function to apply filtering on a substack of images
@delayed
def filter_substack(substack, percentile=75):
    # Ensure the substack is a numpy array
    substack_np = np.array(substack)
    # Return the percentile across the specified axis
    return np.percentile(substack_np, percentile, axis=0).astype(np.uint8)

gray_images = [convert_gray(img) for img in images]
# Convert the list of delayed operations back into a Dask array
gray_images = da.stack([da.from_delayed(d, shape=(height,width), dtype=np.uint8) for d in gray_images], axis=0)

# Beautify frames
pp_images = [beautify_frame(gray_img) for gray_img in gray_images]
pp_images = da.stack([da.from_delayed(d, shape=(height,width), dtype=np.uint8) for d in pp_images], axis=0)

# filter substack
filter_length = 100
percentile = 90
filtered_images = [filter_substack(pp_images[max(0,i-floor(filter_length/2)):i + ceil(filter_length/2)], percentile) for i in range(0, pp_images.shape[0])]
filtered_images = da.stack([da.from_delayed(d, shape=(height,width), dtype=np.uint8) for d in filtered_images], axis=0)

# Trigger the computation
filtered_images_computed = filtered_images.compute()

# Check the result
#print(gray_images_computed)

# We now apply the preprocessing to the images
# Apply gaussian filtering to the stack of images
#smoothed = dask_image.ndfilters.gaussian_filter(images, sigma=[1, 1])
#print(smoothed)

#pp_images = beautify_frame_dask(images)

In [ ]:
# Function to apply filtering on a substack of images
def filter_substack(stack, i, percentile=75, filter_length=100):
    # Create a substack of images for filtering
    substack = stack[i:i + filter_length]
    # Return the percentile across the specified axis
    return da.percentile(substack, percentile).astype(np.uint8)

def preprocess_image(img):
    # Apply preprocessing on the image
    img = beautify_frame(img)
    return img

# Main script
def process_images(image_folder, filter_length=100, percentile=75):
    # List of image files
    files = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))
    # Limit to first 1000 files
    files = files[:1000]
    print(f"Found {len(files)} images")

    # Read the first image to get its dimensions
    first_image = cv2.imread(files[0], cv2.IMREAD_GRAYSCALE)
    height, width = first_image.shape[:2]

    # Do a lazy storage of the images in Dask Arrays
    images = [da.from_array(read_image(f)) for f in files]
    pp_images = [preprocess_image(img) for img in images]

    # Load images as a Dask array (lazy loading) with the dimensions of the first image
    stack = da.stack(pp_images, axis=0)

    # Determine the number of images
    n_frames = len(files)
    print(f"Shape of the stack: {stack.shape}")

    # Apply the filter_substack function in parallel using Dask's delayed for valid indices
    results = [filter_substack(stack, i, percentile, filter_length) for i in range(n_frames - filter_length + 1)]

    # Compute the results
    final_results = dask.compute(*results)

    # Convert the results to a NumPy array
    final_stack = np.array(final_results)

    # Save the result as a TIFF file
    output_filename = "Results/Cyril/filtered_images.tif"
    tifffile.imwrite(output_filename, final_stack)

    print(f"Filtered images saved as {output_filename}")

# Path to your folder containing images
image_folder = "/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/h1r1_1minute/"

# Call the function to process images
process_images(image_folder, filter_length=100, percentile=90)


### Same code but without parallelisation

In [ ]:
import numpy as np
import os, tifffile
from tqdm import tqdm
from skimage.io import imread
import glob
from PIL import Image
from math import floor, ceil
import cv2
import sys
sys.path.append('../../Preprocessing')
from preproc import beautify_frame


# Function to apply filtering on a substack of images
def filter_substack(stack, i, percentile=75, filter_length=100):
    # Create a substack of images for filtering
    substack = stack[i-floor(filter_length/2):i + ceil(filter_length/2)]
    # Return the percentile across the specified axis
    return np.percentile(substack, percentile, axis=0).astype(np.uint8)

# Path to your folder containing images
image_folder = "/Users/cyrilmonette/Desktop/EPFL 2018-2026/PhD - Mobots/data/24.09-24.10_observation_OH/Images/sample_imaging_center/"

files = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))
# Limit to first 1000 files
files = files[:105]
print(f"Found {len(files)} images")

# Read all images using OpenCV with a progress bar and convert to grayscale
image_list = []
for file in tqdm(files, desc="Reading images"):
    img = cv2.imread(file)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_list.append(gray_img)

# Preprocess images using the beautify_frame function
preprocessed_images = []
for img in tqdm(image_list, desc="Preprocessing images"):
    new_img=beautify_frame(img)
    preprocessed_images.append(new_img)

# Stack all the images into a 3D numpy array (depth, height, width)
image_stack = np.stack(preprocessed_images, axis=0)

# For each image, call filter_substack function
filtered_images = []
filter_length = 100
for i in tqdm(range(len(preprocessed_images)), desc="Filtering images"):
    if i < floor(filter_length/2) or i >= len(preprocessed_images) - ceil(filter_length/2):
        continue
    filtered_image = filter_substack(image_stack, i, 95, filter_length)
    filtered_images.append(filtered_image)

# Stack all the filtered images into a 3D numpy array (depth, height, width)
filtered_stack = np.stack(filtered_images, axis=0)

# Save it as tif
output_filename = "Results/Cyril/filtered_images.tif"
tifffile.imwrite(output_filename, filtered_stack)